# Visual Auditor Package Demo (Customer Churn Prediction Dataset)
Author: David Munechika (david.munechika@gatech.edu)

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import visual_auditor
from visual_auditor import SliceFinder

In [2]:
# Load Customer Churn Prediction dataset
customer_churn = pd.read_csv(
    "data/customer_churn_prediction.csv",
    names=[
        "State", "Account Length", "Area Code", "International", "Voicemail Plan", "Voicemail Messages",
        "Day Minutes", "Day Calls", "Day Charge", "Eve Minutes", "Eve Calls", "Eve Charge", "Night Minutes",
        "Night Calls", "Night Charge", "Intl Minutes", "Intl Calls", "Intl Charge", "CustServ Calls", "Churn"],
        engine='python',
        na_values="NA")

# Drop NA values
customer_churn = customer_churn.dropna()

# Bin numerical features
customer_churn["Account Length"] = pd.cut(customer_churn["Account Length"], 3, labels=["0", "1", "2"])
customer_churn["Voicemail Messages"] = pd.cut(customer_churn["Voicemail Messages"], 3, labels=["0", "1", "2"])
customer_churn["Day Minutes"] = pd.cut(customer_churn["Day Minutes"], 3, labels=["0", "1", "2"])
customer_churn["Eve Minutes"] = pd.cut(customer_churn["Eve Minutes"], 3, labels=["0", "1", "2"])
customer_churn["Night Minutes"] = pd.cut(customer_churn["Night Minutes"], 3, labels=["0", "1", "2"])
customer_churn["Intl Minutes"] = pd.cut(customer_churn["Intl Minutes"], 3, labels=["0", "1", "2"])
customer_churn["Day Charge"] = pd.cut(customer_churn["Day Charge"], 3, labels=["0", "1", "2"])
customer_churn["Eve Charge"] = pd.cut(customer_churn["Eve Charge"], 3, labels=["0", "1", "2"])
customer_churn["Night Charge"] = pd.cut(customer_churn["Night Charge"], 3, labels=["0", "1", "2"])
customer_churn["Intl Charge"] = pd.cut(customer_churn["Intl Charge"], 3, labels=["0", "1", "2"])
customer_churn["Day Calls"] = pd.cut(customer_churn["Day Calls"], 3, labels=["0", "1", "2"])
customer_churn["Eve Calls"] = pd.cut(customer_churn["Eve Calls"], 3, labels=["0", "1", "2"])
customer_churn["Night Calls"] = pd.cut(customer_churn["Night Calls"], 3, labels=["0", "1", "2"])
customer_churn["Intl Calls"] = pd.cut(customer_churn["Intl Calls"], 3, labels=["0", "1", "2"])
customer_churn["CustServ Calls"] = pd.cut(customer_churn["CustServ Calls"], 3, labels=["0", "1", "2"])

# Encode categorical features
encoders = {}
for column in customer_churn.columns:
    if customer_churn.dtypes[column] == np.object:
        le = LabelEncoder()
        customer_churn[column] = le.fit_transform(customer_churn[column])
        encoders[column] = le
        print(column, le.classes_, le.transform(le.classes_))

# Separate Target values
X, y = customer_churn[customer_churn.columns.difference(["Churn"])], customer_churn["Churn"]

# Train a classifier
rfc = RandomForestClassifier(max_depth=5, n_estimators=10)
rfc.fit(X, y)

State ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50]
Area Code ['area_code_408' 'area_code_415' 'area_code_510'] [0 1 2]
International ['no' 'yes'] [0 1]
Voicemail Plan ['no' 'yes'] [0 1]
Churn ['no' 'yes'] [0 1]


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [3]:
# Generate slices
sf = SliceFinder(rfc, (X, y))
recommendations = sf.find_slice(k=20, epsilon=0.2, degree=2, max_workers=4)

for s in recommendations:
    print ('\n=====================\nSlice description:')
    for k, v in list(s.filters.items()):
        values = ''
        if k in encoders:
            le = encoders[k]
            for v_ in v:
                values += '%s '%(le.inverse_transform(v_)[0])
        else:
            for v_ in sorted(v, key=lambda x: x[0]):
                if len(v_) > 1:
                    values += '%s ~ %s'%(v_[0], v_[1])
                else:
                    values += '%s '%(v_[0])
        print ('%s:%s'%(k, values))
    print ('---------------------\neffect_size: %s'%(s.effect_size))
    print ('---------------------\nmetric: %s'%(s.metric))
    print ('size: %s'%(s.size))

degree 1
crossing
effect size filtering
degree 2
crossing
effect size filtering
sorting

Slice description:
Day Charge:2 
---------------------
effect_size: 0.4536932508825811
---------------------
metric: 0.5120366673493361
size: 671

Slice description:
Day Minutes:2 
---------------------
effect_size: 0.4536932508825811
---------------------
metric: 0.5120366673493361
size: 671

Slice description:
International:yes 
---------------------
effect_size: 0.472087502928924
---------------------
metric: 0.5674819251427097
size: 396

Slice description:
CustServ Calls:1 
---------------------
effect_size: 0.5751243243975577
---------------------
metric: 0.591584547291761
size: 318

Slice description:
State:TX 
Voicemail Messages:0 
---------------------
effect_size: 0.25739088520451425
---------------------
metric: 0.4602791575069781
size: 77

Slice description:
State:TX 
Voicemail Plan:no 
---------------------
effect_size: 0.25739088520451425
---------------------
metric: 0.460279157506978

In [4]:
visual_auditor.find_slices_and_visualize(rfc, (X, y), k=100, epsilon=0.2, degree=2, max_workers=4)

<iframe
 srcdoc="<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#000000"/><meta name="description" content="Interactive Scalable Auditing of Model Biases and Vulnerabilities with Interpretable Mitigation"/><link rel="apple-touch-icon" href="/logo192.png"/><link rel="manifest" href="/manifest.json"/><title>Visual Auditor</title><style>body{margin:0;font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Oxygen,Ubuntu,Cantarell,"Fira Sans","Droid Sans","Helvetica Neue",sans-serif;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;background:#fff}code{font-family:source-code-pro,Menlo,Monaco,Consolas,"Courier New",monospace}*{scrollbar-width:thin;scrollbar-color:hsla(0,0%,60.8%,.5) transparent}::-webkit-scrollbar{width:5px}::-webkit-scrollbar-track{background:0 0}::-webkit-scrollbar-thumb{background-color:hsla(0,0%,60.8%,.5);border-radius:20px;border:transparent}.nav-container{text-align:center;z-index:2}.settings{margin:1rem}.settings:hover{opacity:1%;cursor:pointer}.left-container{text-align:center;padding:.5rem}.MuiFormControlLabel-label{color:#5f5f5f;font-weight:700}h1{color:#5f5f5f}h2{font-size:1.25rem;padding:.5rem}h2,p{color:#5f5f5f;font-weight:300}p{text-align:left;font-size:1.1rem;padding:0 1rem}.similar-slices-list{font-size:1rem;color:#5f5f5f;font-weight:300;text-align:left}.similar-slice{padding:.25rem 0}.thin{font-weight:300}.right-container{text-align:center;padding:.5rem}.main-container{padding:1rem;text-align:center;overflow-x:scroll;background-color:#fff}img{margin-left:10rem;margin-right:10rem;width:60rem}.hull{fill:rgba(184,125,0,.705);stroke:rgba(184,125,0,.705);opacity:.5;stroke-width:32px;stroke-linejoin:round}.link{stroke:#000;stroke-width:1.5px}.node{cursor:move;fill:#ccc;stroke-width:1.5px}.node.fixed{fill:red}</style></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function r(r){for(var n,i,l=r[0],a=r[1],f=r[2],c=0,s=[];c<l.length;c++)i=l[c],Object.prototype.hasOwnProperty.call(o,i)&&o[i]&&s.push(o[i][0]),o[i]=0;for(n in a)Object.prototype.hasOwnProperty.call(a,n)&&(e[n]=a[n]);for(p&&p(r);s.length;)s.shift()();return u.push.apply(u,f||[]),t()}function t(){for(var e,r=0;r<u.length;r++){for(var t=u[r],n=!0,l=1;l<t.length;l++){var a=t[l];0!==o[a]&&(n=!1)}n&&(u.splice(r--,1),e=i(i.s=t[0]))}return e}var n={},o={1:0},u=[];function i(r){if(n[r])return n[r].exports;var t=n[r]={i:r,l:!1,exports:{}};return e[r].call(t.exports,t,t.exports,i),t.l=!0,t.exports}i.m=e,i.c=n,i.d=function(e,r,t){i.o(e,r)||Object.defineProperty(e,r,{enumerable:!0,get:t})},i.r=function(e){"undefined"!=typeof Symbol&&Symbol.toStringTag&&Object.defineProperty(e,Symbol.toStringTag,{value:"Module"}),Object.defineProperty(e,"__esModule",{value:!0})},i.t=function(e,r){if(1&r&&(e=i(e)),8&r)return e;if(4&r&&"object"==typeof e&&e&&e.__esModule)return e;var t=Object.create(null);if(i.r(t),Object.defineProperty(t,"default",{enumerable:!0,value:e}),2&r&&"string"!=typeof e)for(var n in e)i.d(t,n,function(r){return e[r]}.bind(null,n));return t},i.n=function(e){var r=e&&e.__esModule?function(){return e.default}:function(){return e};return i.d(r,"a",r),r},i.o=function(e,r){return Object.prototype.hasOwnProperty.call(e,r)},i.p="/";var l=this["webpackJsonpvisual-auditor"]=this["webpackJsonpvisual-auditor"]||[],a=l.push.bind(l);l.push=r,l=l.slice();for(var f=0;f<l.length;f++)r(l[f]);var p=a;t()}([])</script><script type="text/javascript">/*! For license information please see 2.d769a9af.chunk.js.LICENSE.txt */
(this["webpackJsonpvisual-auditor"]=this["webpackJsonpvisual-auditor"]||[]).push([[2],[function(e,t,n){"use strict";e.exports=n(234)},function(e,t,n){"use strict";e.exports=n(240)},function(e,t,n){"use strict";function r(){return(r=Object.assign||function(e){for(var t=1;t<arguments.length;t++){var n=argume

In [ ]:
# Alternative use - separate slice-finding and visualization functions
sf = SliceFinder(rfc, (X, y))
sf.find_slices_and_visualize(k=20, epsilon=0.2, degree=1, max_workers=4)

visual_auditor.visualize()